Plotly: Vem com ferramentas para criar o dashboard e construir gráfico

Dash Bootstrap Components permite inserir temas que ajudam no visual

Dados tirados de covid.saude.gov.br, onde é possível obter informações atuais do Brasil.

Temos brazil_geo_.json que tem informações geográficas do Brasil.

Temos os assets que darão a logo e o estilo pré configurado.




# Dataset

Informações do Brasil, Estados e Municipios, com 2 milhões de linhas

Possui colunas de região, estado, município, códigos do município, datas, quantidades de casos, óbitos, etc.

Quando os dados se referem ao Brasil, não vemos informações de estado e município. Assim como dados de estados e municípios não possuem dados do Brasil. Devemos entender isso para conseguir separá-los. 

# Formatação



In [4]:
!pip install dash jupyter_dash dash_bootstrap_components

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
import dash #gerencia nosso Dashboard, criando o servidor e layout
from dash import dcc #seleção de data, dropdowns
from dash import html #permite inserção de textos, divs, caixas em html
from dash.dependencies import Input, Output 
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash

import plotly.express as px #criação de gráficos mais fáceis de alto nível de forma básica
import plotly.graph_objects as go #permitem maior controle que o px

import numpy as np
import pandas as pd
import json

Os dados disponibilizados pelo governo estão contidos em partes, 2 partes por ano, para cada semestre. Para juntá-los é necessário o uso do concat, do pandas, conseguindo assim a união de todos em um só.



In [6]:
df1 = pd.read_csv("2020_1.csv", sep=';')
df2 = pd.read_csv("2020_2.csv", sep=';')
df3 = pd.read_csv("2021_1.csv", sep=';')
df4 = pd.read_csv("2021_2.csv", sep=';')
df5 = pd.read_csv("2022_1.csv", sep=';')
df6 = pd.read_csv("2022_2.csv", sep=';')
df7 = pd.read_csv("2023_1.csv", sep=';')

# Combine todos os dataframes em um único dataframe
df = pd.concat([df1, df2, df3, df4, df5, df6, df7], ignore_index=True)

O nosso maior interesse é obter informações sobre o país inteiro e de cada estado, além de reduzir os dados em uso. Para isso iremos filtrar nossa tabela principal, extraindo informações nacionais e estaduais, através de uma lógica. Primeiramente vamos extrair os dados estaduais, para isso pegamos do dataframe original todas as colunas que possuem "estado" não NA (~.isna()) e "codmun" NA (.isna()). Depois vamos pegar informações nacionais apenas, onde a coluna "regiao" é "Brasil", que automaticamente não carrega informações estaduais e municipais. Para simplificar nossa vida vamos criar um arquivo csv dos novos dataframes criados.

In [7]:
# Queremos só informação dos estados
df_estados = df[(~df["estado"].isna()) & (df["codmun"].isna())]

# Queremos só informação do brasil
df_brasil = df[df['regiao'] == 'Brasil']

# Criar um arquivo para esses dados
df_estados.to_csv("df_estados.csv")
df_brasil.to_csv("df_brasil.csv")

In [8]:
df_estados = pd.read_csv('df_estados.csv')
df_brasil = pd.read_csv('df_brasil.csv')

Agora vamos definir nossas variáveis padrões, só para conseguir definir inicialmente nossos gráfico e algumas funções. Elas serão:
- df_estados_: informações estaduais do dia 10/05/2020, para o gráfico mapa

- df_data: informações estaduais do RJ, para o graph_objects

- date: dia 10/05/2020

- location: estado de Roraima

- CENTER_LAT e CENTER_LON: coordenadas para centrar nosso mapa na tela

In [9]:
df_estados_ = df_estados[df_estados["data"] == '2020-05-10']
df_data = df_estados[df_estados["estado"]=="RJ"] 
date = "2020-05-10"
location = "RO"
CENTER_LAT = -14.272572694355336
CENTER_LON = -51.25567404158474

In [10]:
select_columns = {"casosAcumulado": "Casos Acumulados",
                  "casosNovos": "Novos Casos",
                  "obitosAcumulado": "Óbitos Totais",
                  "obitosNovos": "Obitos por dia"}

# Arquivo geo.json

Usado para delimitar territórios estaduais do Brasil. Ele contém as informações de latitude e longitude de cada divisa dos estados. 

In [11]:
estados_brasil = json.load(open("brazil_geo.json",'r'))

In [12]:
estados_brasil.keys()

dict_keys(['type', 'features'])

In [13]:
estados_brasil["type"]

'FeatureCollection'

In [14]:
estados_brasil["features"][0].keys()

dict_keys(['type', 'id', 'properties', 'geometry'])

É uma lista de dicionários com chaves type, id, properties e geometry

O id é uma chave que será usada para criar o mapa, que irá casar essas informações territoriais às informações dos dados, havendo valores em comuns entre os dois, colorindo os estados a partir das informações.

A geometria é composta por pontos de latitude e longitude.


# Dashboard

Ele estará contido no app

Iremos usar o external_stylesheets, para trazer uma estrutura de título, cores e fontes prontas, melhor do que a padrão. Vamos usar o tema CYBORG

In [15]:
app = JupyterDash(__name__, external_stylesheets = [dbc.themes.CYBORG])

## Figura

Vai ter nosso gráfico do mapa onde será colocada no dashboard através de uma figure

choropleth tem divisões territoriais

o mapblox cria gráficos com configurações modernas, podendo inserir diversas informações bacanas.

precisamos passar o dataframe, coluna com informações que irá casar com o id do geo.json, pintando através de color a coluna do dataframe, o center será a latitude e longitude que o gráfico estará centrado (para isso iremos usar uma coordenada centrada no Brasil), o arquivo geo.json, a paleta de cores, opacidade e finalmente o hover_data, que é um dicionário com colunas que serão apresentadas ao passar o cursor do mouse nos estados.

##Choropleth_mapbox

Ele vai receber as seguintes especificações:

- df: data frame que será usado para o gráfico. Para o mapa escolhemos o df_estados, que possui informações de cada estado. 

- geojson: apresenta a geometria de uma região geográfica no mapa, onde será preenchida por cores. No nosso caso, no Brasil cada estado será colorido, então vamos fornecer o arquivo geojson dessas delimitações. 

- locations: qual coluna com informações geográficas será usada para associar os limites geográficos das regiões do mapa. Iremos usar a coluna "estado", pois cada estado será colorido diferentemente.
  
- color: qual coluna atribuirá cor ao location. Cada estado será colorido de acordo com uma coluna de dados, que será a de "casosNovos"

- hover_data: lista de colunas com valores True e False, que permite adicionar informações adicionais aos dados do mapa quando o cursor do mouse passa por cima de uma determinada área. Queremos que ao passar o mouse, seja mostrado a quantidade de casos acumulados, novos casos, obitos e obitos diário.

- color_continuous_scale: escala de cores usada nas regiões geográficas do mapa. Usaremos o "Redor", que mostrará em diferentes tons de vermelho os dados variados nos estados do país.

- opacity: transparência das regiões geográficas. Usaremos um valor não muito alto para que seja possível enxergar o conteúdo de cada estado.

- zoom: nível de zoom do mapa, varia de 0 a 20. Um valor agradável é 4, suficiente para mostrar o país inteiro.

- center: ponto central do mapa, composto por latitude e longitude, onde é possível definir o foco para uma visualização agradável.

O layout será atualizado:

- paper_bgcolor: a cor de fundo da área onde o conteúdo do gráfico está. A paleta utilizada é uma com cor escura, minimalista e discreta, causando contraste com a cor avermelhada.

- autosize: preenchimento automático do gráfico de acordo com o espaço na tela para True.

- margin: espaço vazio entre as bordas do gráfico e o conteúdo dele.

- showlegend: exibir legenda, True para sim e False para não.

- mapbox_style: é o estilo do mapa. Usamos o carto-darkmatter, para um estilo escuro da cartografia mundial.

##Scatter

Ele vai receber as seguintes especificações:

- layout: Vamos usar o plotly_dark, que permite paletas de cores escuras ao gráfico, destacando elementos visuais e facilitando a observação.

- x e y: Eixo x e y do gráfico. O x será o tempo e y outros dados.

O layout será atualizado:

- paper_bgcolor: cor de fundo da área externa do gráfico. Usaremos a mesma.

- plot_bgcolor: cor de fundo da área do gráfico.Usaremos a mesma.

- autosize: preenchimento automático do gráfico de acordo com o espaço na tela para True.

- margin: espaço vazio entre as bordas do gráfico e o conteúdo dele

In [16]:
#criar gráfico de mapa
fig = px.choropleth_mapbox(df_estados_, geojson=estados_brasil, locations='estado', color = 'casosNovos',
                           hover_data = {"casosAcumulado": True, "casosNovos":True, "obitosNovos":True, "estado":True},
                           color_continuous_scale='Redor', opacity=0.4, zoom=4, 
                           center = {"lat": -16.95, "lon": -47.78})

fig.update_layout(
    paper_bgcolor = "#242424", #paleta de cores
    autosize=True, #gráfico preenche automaticamente o tamanho na caixa contida
    margin = go.Margin(l=0, r=0, t=0, b=0), 
    showlegend = False,
    mapbox_style='carto-darkmatter'
)

#criar gráfico de linha

fig2 = go.Figure(layout={'template':'plotly_dark'})
fig2.add_trace(go.Scatter(x=df_data["data"], y=df_data['casosAcumulado']))
fig2.update_layout(
    paper_bgcolor='#242424',
    plot_bgcolor='#242424',
    autosize = True,
    margin = dict(l=10,r=10,t=10,b=10) #queremos uma borda entre a fig2
)

/usr/local/lib/python3.9/dist-packages/plotly/graph_objs/_deprecations.py:405: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




#Layout

Vamos usar o layout pronto da biblioteca dbs, onde podemos usar um sistema com 1 container com linhas e colunas, especificando esses tamanhos de acordo com a tela.

Teremos container, row e call. Basicamente temos 2 colunas, mas cada coluna precisa de uma linha para o container

A coluna da esquerda terá dados, tabela e divs.

A coluna da direita será o mapa do Brasil

Nosso Layout será criado, ele será formado por duas  grandes linhas (Row), uma no topo e a outra abaixo, contendo nela três colunas (Col). Estas três possuem comprimentos ajustáveis pela Row, onde a soma deles não pode passar de 12, comprimento total da Row, possibilitando assim uma personalização dos tamanhos das colunas.

O Container terá todo o nosso layout, formado por duas grandes linhas e 3 colunas contidas em uma delas. Nossa ideia é criar no Dashboard uma separação entre a apresentação dos dados e o mapa.

Nosso esquema de layout é o seguinte:

- Container: define uma região retangular na página, contendo outros componentes, como gráficos, tabelas e textos.

  - Row1: agrupará outros componentes horizontalmente, como uma linha, colunas e outras Row.

    - Col1: agrupa outros componentes verticalmente na Row1, como uma coluna

      - Img: Carregar uma imagem para a coluna

      - H5: Inserir um texto de tamanho H5

      - Button: Inserir um botão que executará alguma ação, determinada pelo callback.

      - P: Elemento paragráfico

      - Div: Cria um contêiner genérico para outros elementos html

        - DatePickerSingle: calendário em janela flutuante.

      - Row1.1: Agora criar uma linha com 3 colunas contendo cartões em cada uma

        - Col1.1
          - Card1.1: exibe informações, gráficos, imagens e outros elementos visuais. 
            - CardBody1.1: adiciona conteúdo ao cartão, como texto, gráficos, tabelas e outros elementos ao corpo do cartão.

      - Div
        - P
        - Dropdown: menu suspenso que exibe uma lista de opções para o usuário selecionar
        - Graph

    - Col2
      - Loading
        - Graph: criação de gráficos interativos 

In [17]:
app.layout = dbc.Container(
    
    dbc.Row([ 
        
        dbc.Col([
            html.Div([
            html.Img(id="logo", src='https://uploaddeimagens.com.br/images/004/196/406/full/notebook_livia.png?1669774882',height=20, style={'object-fit': 'contain'}),
            html.H5("Evolução COVID-19"),
            dbc.Button(children="BRASIL",color="primary", id="location-button",size="lg")
        ], style={}),  

            html.P("Informe a data na qual deseja obter informações", style={"margin-top":"40px"}),

            html.Div(id="div-test",children =[
                dcc.DatePickerSingle(
                    id="date-picker",
                    min_date_allowed=df_brasil["data"].min(),
                    max_date_allowed=df_brasil["data"].max(),
                    initial_visible_month = df_brasil["data"].min(),
                    date=df_brasil["data"].max(),
                    display_format="DD/MM/YYYY",
                    style={"border":"0px solid black"}
                ) #objeto que nos permite escolher data

            ]),

            dbc.Row([
                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span("Casos recuperados"),
                            html.H3(style={"color": "#adfc92"}, id="casos-recuperados-text"),
                            html.Span("Em acompanhamento"),
                            html.H5(id="em-acompanhamento-text"),
                          
                        ])
                    ], color='light', outline=True, style={"margin-top": "10px",
                                                           "box-shadow": "0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)",
                                                           "color": "#FFFFFF"})
                ], md=4), #largura total que a coluna ocupa, o total são 12

                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span("Casos confirmados totais"),
                            html.H3(style={"color": "#389fd6"}, id="casos-confirmados-text"),
                            html.Span("Novos casos na data"),
                            html.H5(id="novos-casos-text"),
                          
                        ])
                    ], color='light', outline=True, style={"margin-top": "10px",
                                                           "box-shadow": "0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)",
                                                           "color": "#FFFFFF"})
                ], md=4),

                dbc.Col([
                    dbc.Card([
                        dbc.CardBody([
                            html.Span("Óbitos confirmados"),
                            html.H3(style={"color": "#DF2935"}, id="obitos-text"),
                            html.Span("Óbitos na data"),
                            html.H5(id="obitos-na-data-text"),
                          
                        ])
                    ], color='light', outline=True, style={"margin-top": "10px",
                                                           "box-shadow": "0 4px 4px 0 rgba(0, 0, 0, 0.15), 0 4px 20px 0 rgba(0, 0, 0, 0.19)",
                                                           "color": "#FFFFFF"})
                ], md=4),
                
            ]),

            html.Div([
                html.P("Selecione que tipo de dado deseja visualizar:", style={"margin-top":"25px"}),
                dcc.Dropdown(id="location-dropdown",
                         options=[{"label": j, "value": i} for i, j in select_columns.items()],
                         value="casosNovos",
                         style={"margin-top":"10px"}
                         ),
                dcc.Graph(id="line-graph", figure=fig2)

            ]),

            
        ], md=5, style={"padding":"25px", "background-color": "#242424"}),
        
        dbc.Col([
            dcc.Loading(id="loading-1", type="default",
                        children = [dcc.Graph(id="choropleth-map",
                         figure=fig, style={"height": "100vh", "margin-right": "10px"})
                        ]
                        
                    )
            
        ], md=7)
    ], class_name='g-0')
, fluid = True)

# Interatividade
Precisamos criar funções que receberão parâmetros dos componentes do dash e retornarão outros valores que serão encaixados em alguns atributos de outros componentes.

Para isso usaremos um decorador especial do dash, o @app.callback, que pede dois parâmetros, o Output e o Input da função a ser criada. Assim o @app.callback será acionado quando qualquer parâmetro do app for alterado, chamando a função para alterar outros parâmetros.

Devemos informar em cada Output e Input primeiramente o component_id e depois o component_property.

Temos 4 callbacks:

- Callback 1: Queremos selecionar uma data e que chame uma função ao modificar a data, para que os valores contidos nos cards sejam alterados, de acordo com a data e localização. Para o Input iremos definir o component_id sendo o "date-picker" e o "component-property" "date". Assim, uma vez que "date" for alterado, o dash automaticamente vai chamar a função abaixo, passando o "date" como parâmetro, pois é o primeiro Input da lista de Inputs. Teremos agora outro Input, o "location-button". Quando seu children for alterado, ou seja, "BRASIL" não for mais "BRASIL", também chamará a mesma função. Agora temos uma função que alterará os valores dos cards de acordo com a data e localização, então quando outras funções a modificarem, essa função será chamada também. Agora, quem será modificado? Iremos alterar todos os html.H3, sendo textos contidos nos cards, compostos por 6 outputs, 2 em cada card. O children desses H3 é definido implicitamente, pois não está escrito lá em cima, então inicialmente nada está escrito abaixo de "Casos confirmados totais", o que será preenchido por um valor devolvido pela função. Então basicamente queremos que nossa função pegue a data e localização, e devolva dados dos estados (df_estados) ou brasil (df_brasil), com os valores respectivos de data e localização de cada df. Se location = "BRASIL" vamos puxar informações de df_brasil, caso contrário, df_estados. Assim definimos para cada ocorrência um df_data_on_date, que será um dataframe contendo dados apenas daquela data em específico. Caso location não seja BRASIL, iremos criar um df_data_on_date, onde só haverá informação da location especificada (um estado) na devida data selecionada (date). Agora temos um detalhe para os valores data_on_date, se nossa location não for Brasil, for algum estado, não haverão dados para "Recuperadosnovos", pois esses dados estão disponíveis apenas para o Brasil. Então, para mostrar ao dashboard que não há essas informações, será preciso criar uma função que apresentará um "-" casos essa coluna for nula (isna()). Se não for nula iremos apenas formatar o valor, para mostrar as separações em ".". Então,  aqui há a alteração da localização pelo location-button, e uma alteração na data, promovendo o uso de dados naquela localização e data. Sendo assim, a variável "location-button" afetará também o 2° e 4° Callbacks, mexendo nos gráficos de barra e linhas e também nos H5, assim como a variável "date-picker" irá afetar o 3° Callback, na hora de preencher os dados do mapa com a devida data. 

- Callback 2: Selecionar o Dropdown (lista com opções) e a localização irá alterar a figure do nosso line-graph, mostrando diferentes dados para diferentes estados. Então criaremos uma função que receberá plot_type (vinda do "value") e location. Se a localização for "BRASIL" criaremos um df_data_on_location e usaremos seus dados, caso contrário, criaremos uma df_data_on_location com dados do estado especificado. Além disso iremos alterar o visual do gráfico dependendo do parâmetro escolhido no Dropdown. Para dados acumulados usaremos gráfico de linhas, e para dados novos usaremos gráfico de barra. Então criaremos uma lista chamada bar_plots, contendo os nomes de cada opção do Dropdown que serão do tipo Barra. Caso nosso plot_type vinda do value do input seja uma string contida na lista bar_plots, iremos atualizar a figura para um gráfico de barra, caso contrário irá virar um gráfico de linha. Assim, para ambos os gráficos, nosso eixo X será o df_data_on_location["Data"], sendo a data até onde ocorreram os dados, e o nosso eixo Y os dados locais (de óbitos ou casos, podendo ser novos ou acumulados). O "location-button" alterado aqui, além de disparar a função desse Callback 2, irá disparar o Callbacks 1, alterando os H5. O "location-dropdown" apenas aplica mudanças nesse Callback, alterando o gráfico de linha ou barras.

- Callback 3: Alterar a data altera o gráfico choropleth, do Brasil, preenchendo cada estado com seus devidos valores em uma data específica, podendo acompanhar os dados de cada estado. Teremos como Input a Data e Output o gráfico, que serão porta de entrada para a nossa função, que criará outro choropleth, com informações de acordo com a data selecionada. O "date" alterado aqui, além de disparar a função desse Callback que altera o mapa, também irá disparar a do Callback 1, alterando os dados em H5 dos cards.

- Callback 4: Clicar em cada estado irá alterar nossos dados e clicar no "location-button" irá resetar nossos dados para o Brasil. Isso ocorrerá através de nossos Inputs, que serão o choropleth-map", onde queremos o "clickData" dele, e o "n_clicks" do "location-button". Definiremos o n_clicks como sendo a quantidade de vezes que o location-button foi clicado. Nossa função recebe o click_data e o n_clicks, ela irá conter uma variável (changed_id) que irá entender qual dos últimos eventos foi clicado, o mapa ou o location-button. Vamos usar uma propriedade do dash que nos permite saber qual foi o última evento que disparou a nossa função, o [p["prop_id"] for p in dash.callback_context.triggered][0]. Esta será uma lista que contém todos os valores que estão aplicando mudanças na função desse Callback, com o [0] conseguimos o click mais atual. Caso o click atual tenha valor (retornaria vazio se o clique fosse em outro local) e seja diferente de location-button.n_clicks, saberemos que o clique foi no mapa. Assim o estado selecionado será definido, retornando o seu nome no location-button, caso contrário, clique no "location-button", retornará Brasil. O "location-button" alterado aqui, além de disparar nossa função desse Callback, irá disparar o Callback 1, alterando os dados H5 dos cards, e o Callback 2, alterando o plot gráfico de linha ou barras. 

In [18]:
@app.callback(
    [ 
      Output("casos-recuperados-text", "children"),
      Output("em-acompanhamento-text", "children"),
      Output("casos-confirmados-text", "children"),
      Output("novos-casos-text", "children"),
      Output("obitos-text", "children"),
      Output("obitos-na-data-text", "children")
    ],
     [Input("date-picker", "date"),
      Input("location-button", "children")]
     )

def display_status(date, location):
  if location == "BRASIL":
    df_data_on_date = df_brasil[df_brasil["data"] == date]
  else:
    df_data_on_date = df_estados[(df_estados["estado"] == location) & (df_estados["data"]==date)]
  
  df_data_on_date["Recuperadosnovos"]
  recuperados_novos = "-" if df_data_on_date["Recuperadosnovos"].isna().values[0] else f'{int(df_data_on_date["Recuperadosnovos"].values[0]):,}'.replace(",",".")
  acompanhamentos_novos = "-" if df_data_on_date["emAcompanhamentoNovos"].isna().values[0] else f'{int(df_data_on_date["emAcompanhamentoNovos"].values[0]):,}'.replace(",",".")
  casos_acumulados = "-" if df_data_on_date["casosAcumulado"].isna().values[0] else f'{int(df_data_on_date["casosAcumulado"].values[0]):,}'.replace(",",".")
  casos_novos = "-" if df_data_on_date["casosNovos"].isna().values[0] else f'{int(df_data_on_date["casosNovos"].values[0]):,}'.replace(",",".")
  obitos_acumulado = "-" if df_data_on_date["obitosAcumulado"].isna().values[0] else f'{int(df_data_on_date["obitosAcumulado"].values[0]):,}'.replace(",",".")
  obitos_novos = "-" if df_data_on_date["obitosNovos"].isna().values[0] else f'{int(df_data_on_date["obitosNovos"].values[0]):,}'.replace(",",".")
  
  return(recuperados_novos,
         acompanhamentos_novos,
         casos_acumulados,
         casos_novos,
         obitos_acumulado,
         obitos_novos)

@app.callback(Output("line-graph", "figure"),
              [
                  Input("location-dropdown", "value"),
                  Input("location-button", "children")
              ])

def plot_line_graph(plot_type,location):
  if location == "BRASIL":
    df_data_on_location = df_brasil.copy()
    
  else:
    df_data_on_location = df_estados[df_estados["estado"] == location]

  bar_plots = ["casosNovos", "obitosNovos"]

  fig2 = go.Figure(layout={"template":"plotly_dark"})
    
  if plot_type in bar_plots:
    fig2.add_trace(go.Bar(x=df_data_on_location["data"], y=df_data_on_location[plot_type]))
    
  else:
    fig2.add_trace(go.Scatter(x=df_data_on_location["data"], y=df_data_on_location[plot_type]))

  fig2.update_layout(
        paper_bgcolor="#242424",
        plot_bgcolor="#242424",
        autosize=True,
        margin=dict(l=10,r=10,b=10,t=10)
    )
  return fig2


@app.callback(
    Output("choropleth-map","figure"),
    [Input("date-picker","date")]
)
def update_map(date):
  df_data_on_states = df_estados[df_estados["data"] == date]

  fig = px.choropleth_mapbox(df_data_on_states, locations = "estado", geojson=estados_brasil,
                             center = {"lat":CENTER_LAT, "lon":CENTER_LON},
                             zoom = 4, color="casosAcumulado", color_continuous_scale="Redor",opacity=0.55,
                             hover_data={"casosAcumulado":True,
                             "casosNovos":True, "obitosNovos":True, "estado":False})
  
  fig.update_layout(paper_bgcolor="#242424",mapbox_style="carto-darkmatter",autosize=True,
                    margin=go.layout.Margin(l=0,r=0,t=0,b=0), showlegend=False)
  
  return fig

@app.callback(
    Output("location-button","children"),
    [Input("choropleth-map", "clickData"),
     Input("location-button","n_clicks")]
)

def update_location(click_data, n_clicks):
  changed_id = [p["prop_id"] for p in dash.callback_context.triggered][0]
  if click_data is not None and changed_id != "location-button.n_clicks":
    state = click_data["points"][0]["location"]
    return "{}".format(state)
  else:
    return "BRASIL"


if __name__ == "__main__":
  app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/



INFO:dash.dash:Dash is running on http://127.0.0.1:8050/



Dash app running on:


<IPython.core.display.Javascript object>

#Interatividade

Executar as funções que irão alterar a tabela

Quando selecionarmos os estados com mouse, queremos que ele filtre os dados e mostre os dados do estado na data atual no gráfico ao lado. Se clicar no botão BRASIL voltar aos dados Brasil. Ao selecionar a data alterar o mapa e os valores. Selecionar entre óbitos, casos confirmados, etc. Diferentes gráficos por tipos de dados.

Isso é feito a partir da criação de funções que recebem parâmetros dos componentes dos dash e que retornarão valores que serão encaixados em atributos de outros componentes.